In [1]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks
import numpy as np 
import cv2 as cv 
import matplotlib as plt 

In [34]:
epochs = 300
batch_size = 64
patience = 20
learning_rate = 0.001
model_path = 'checkpoints/model.keras'

In [35]:
exists = os.path.exists(model_path)

model = models.load_model(model_path) \
    if exists \
        else models.Sequential([
            layers.Resizing(20, 20),
            layers.Rescaling(scale=1./127.5, offset=-1),
            layers.RandomRotation((-0.1, 0.1)),
            layers.Conv2D(256, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(256, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(1024,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(62,
                activation = 'sigmoid',
                kernel_initializer = initializers.RandomNormal()
            )
        ])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
        learning_rate = learning_rate
        ),
        loss = losses.SparseCategoricalCrossentropy(),
        metrics = [ 'accuracy' ]
    )

In [30]:
train = utils.image_dataset_from_directory(
    "Img", # Change path
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

test = utils.image_dataset_from_directory(
    "Img", # Change path
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

Found 3410 files belonging to 62 classes.
Using 2728 files for training.
Found 3410 files belonging to 62 classes.
Using 682 files for validation.


In [39]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/300


86/86 [==============================] - 4s 41ms/step - loss: 0.4223 - accuracy: 0.8508 - val_loss: 0.8729 - val_accuracy: 0.8255
Epoch 2/300
86/86 [==============================] - 4s 43ms/step - loss: 0.4517 - accuracy: 0.8490 - val_loss: 0.7965 - val_accuracy: 0.8314
Epoch 3/300
86/86 [==============================] - 3s 39ms/step - loss: 0.4317 - accuracy: 0.8552 - val_loss: 0.8640 - val_accuracy: 0.8284
Epoch 4/300
86/86 [==============================] - 3s 39ms/step - loss: 0.4361 - accuracy: 0.8537 - val_loss: 0.8693 - val_accuracy: 0.8314
Epoch 5/300
86/86 [==============================] - 4s 40ms/step - loss: 0.4079 - accuracy: 0.8508 - val_loss: 0.9102 - val_accuracy: 0.8182
Epoch 6/300
86/86 [==============================] - 4s 43ms/step - loss: 0.3941 - accuracy: 0.8651 - val_loss: 0.9363 - val_accuracy: 0.8270
Epoch 7/300
86/86 [==============================] - 4s 45ms/step - loss: 0.3867 - accuracy: 0.8614 - val_loss: 0.8574 - val_accuracy: 0.8299
Epoch 8/300
86/86 